In [1]:
import sys
import pandas as pd
from difflib import get_close_matches

In [2]:
filename1 = "movie_list.txt"
filename2 = "movie_ratings.csv"
outputfile = "output"  #output.csv

In [3]:
movies = pd.read_table(filename1,header=None, names=["title"])
# ratings = pd.read_table(filename2,)
ratings = pd.Series(open(filename2).readlines())

In [4]:
def CleanRate(rating):
    rating = rating.replace('\n', '')
    return rating.rsplit(',',1)

In [5]:
ratings = ratings.apply(CleanRate)
ratings = ratings[1:]

In [6]:
rate_data = pd.DataFrame.from_items(zip(ratings.index,ratings.values)).T


In [7]:
rate_data = rate_data.rename(columns={0:'title', 1:'rating'})

In [8]:
join_str = ''
def MatchName(name):
    res = get_close_matches(name, movies['title'], n=1)
    return join_str.join(res)

In [9]:
rate_data['title'] = rate_data['title'].apply(MatchName)

In [10]:
# print(rate_data[:15])

In [11]:
clean_data = rate_data[rate_data['title'].str.len() != 0]

In [12]:
clean_data['rating'] = clean_data['rating'].apply(lambda x: int(x))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
grouped_data = clean_data.groupby('title')

In [14]:
final_data = grouped_data.aggregate('mean').reset_index()

In [15]:
final_data['rating'] = final_data['rating'].round(2)

In [16]:
final_data = final_data.sort_values('title', axis=0)

                                 title  rating
0                     28 Days Later...    7.18
1                       A Time to Kill    7.92
2                            Anomalisa    7.50
3                 Central Intelligence    6.88
4                   Dead Poets Society    8.33
5                           Detachment    8.14
6                    Dial M for Murder    8.12
7                    Dog Day Afternoon    8.60
8                            El cuerpo    8.44
9                              Everest    7.07
10              Everybody Wants Some!!    6.88
11                        Garden State    8.67
12                   God Bless America    7.50
13                  Heaven Is for Real    6.12
14                    Hitman: Agent 47    5.50
15      House at the End of the Street    5.95
16              I Spit on Your Grave 2    6.23
17                        Intouchables    8.93
18                Jûsan-nin no shikaku    8.00
19                       Love Actually    8.00
20           

In [17]:
final_data.to_csv(outputfile,index = False)